In [4]:
# import libraries
import numpy as np
import pandas as pd

In [22]:
# import data
dir = 'C:\\Users\\Lenovo\\PycharmProjects\\Kaggle\\Project2_Expedia\\'
train_set = pd.read_csv(dir + 'train.csv', nrows=10)
test_set = pd.read_csv(dir + 'test.csv')
dest_set  = pd.read_csv(dir + 'destinations.csv')

ParserError: Error tokenizing data. C error: out of memory

In [19]:
train_set.head().shape

(5, 24)

In [6]:
file = open(dir + 'train.csv')
file.readline()

'date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster\n'

In [7]:
file.readline()

'2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,9,2014-08-27,2014-08-31,2,0,1,8250,1,0,3,2,50,628,1\n'